# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for audio mosaicing. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. If you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [19]:
import os
import pandas as pd
import numpy as np
import freesound
import essentia
import essentia.standard as estd
from IPython.display import display

FILES_DIR = 'files'  # Place where to store the downloaded files. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File to store the metadata of the sound collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store
TARGET_PATH = 'StarWars.mp3'

if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [16]:
# Step 1: Define the file path where your API key is stored
file_path = "api_key.txt"

try:
    # Step 2: Open the file in read mode
    with open(file_path, 'r') as file:
        # Step 3: Read the contents of the file
        api_key = file.read().strip()  # Use strip() to remove any leading/trailing whitespace
        freesound_client = freesound.FreesoundClient()
        freesound_client.set_token(api_key)
    # Step 4: The file is automatically closed when exiting the 'with' block
except FileNotFoundError:
    print("File not found. Please check the file path.")
except IOError as e:
    print(f"Error reading the file: {e}")

In [17]:
# Define some util functions

def query_freesound(query, filter, sort, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results,
        sort = sort,
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

def get_key_from_target(target_path):
    loader = estd.MonoLoader(filename=target_path)
    audio = loader()
    key_algo = estd.KeyExtractor()
    key, scale, key_strength = key_algo(audio)
    print(key_strength)
    return '{0} {1}'.format(key, scale)

In [18]:
# Build collection of sounds

key = get_key_from_target(TARGET_PATH)
print(key)
# The collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'lightsaber',
        'filter': None,
        'num_results': 20,
        'sort': 'score',
    },
    {
        'query': 'laser',
        'filter': None,
        'num_results': 20, 
        'sort': 'score',
    },
    {
        'query': 'r2d2',
        'filter': None,
        'num_results': 20, 
        'sort': 'score',
    },
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['sort'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of the sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of the DataFrame (the metadata of the source collection)
display(df)

0.8979130983352661
Bb major
Saved DataFrame with 60 entries! dataframe.csv


,name,username,license,tags,freesound_id,path
0,Before-After remix of 78674__joe93barlow__on0-...,Timbre,https://creativecommons.org/licenses/by-nc/4.0/,"[78674, ignite, joe93barlow, laser, lazer, lig...",115591,files/115591_1015240-hq.ogg
1,on0.wav,joe93barlow,http://creativecommons.org/publicdomain/zero/1.0/,"[lightsaber, star, wars]",78674,files/78674_1083679-hq.ogg
2,Lightsaber #1,danlucaz,http://creativecommons.org/publicdomain/zero/1.0/,"[starwars, series, energy, film, swings, saber...",517746,files/517746_9996727-hq.ogg
3,Lightsaber Ignition,pip_,http://creativecommons.org/publicdomain/zero/1.0/,"[sound, star, wars, lightsaber, synth, ignitio...",557194,files/557194_6737463-hq.ogg
4,Lightsaber Whoosh,h2p34,http://creativecommons.org/publicdomain/zero/1.0/,"[Hit, Whoosh, Lightsaber, Sword, Electro, Elec...",650564,files/650564_9534185-hq.ogg
5,lightsaber4.mp3,gyzhor,http://creativecommons.org/publicdomain/zero/1.0/,"[ignition, light, lightsaber, saber]",47126,files/47126_383523-hq.ogg
6,lightsaber1.mp3,gyzhor,http://creativecommons.org/publicdomain/zero/1.0/,"[light, lightsaber, mixdown, saber]",47123,files/47123_383523-hq.ogg
7,lightsaber2.mp3,gyzhor,http://creativecommons.org/publicdomain/zero/1.0/,"[extinguish, light, lightsaber, saber]",47124,files/47124_383523-hq.ogg
8,lightsaber5.mp3,gyzhor,http://creativecommons.org/publicdomain/zero/1.0/,"[idle, light, lightsaber, loop, saber]",47127,files/47127_383523-hq.ogg
9,lightsaber3.mp3,gyzhor,http://creativecommons.org/publicdomain/zero/1.0/,"[light, lightsaber, saber, swing]",47125,files/47125_383523-hq.ogg
